# Dataset Validation
---

## Sanity Check

In [7]:
import json

def load_config(file_path: str = "./config.json"):
    with open(file_path) as config_file:
        data = json.load(config_file)
    return data

config = load_config("../config.json")
DBNAME = config.get("DBNAME")
HOSTNAME = config.get("HOSTNAME")
USER = config.get("USER")
PASS = config.get("PASS")
SCHEMA = config.get("SCHEMA")

In [8]:
# Basic 
import sys
import numpy as np
import scipy as sp
import pandas as pd

# SQL Engine
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

# Profiling process
from tqdm import tqdm

# Warning problems in notebook
import warnings
warnings.filterwarnings('ignore')

# Reporting result
import sweetviz as sv
from dataprep.eda import create_report

In [9]:
# Load data

# Create an engine instance
alchemyEngine = create_engine(
    f'postgresql+psycopg2://{USER}:{PASS}@{HOSTNAME}/{DBNAME}', pool_recycle=3600)

# Connect to PostgreSQL server
conn = alchemyEngine.connect()

schema = SCHEMA

---
__**An Overview of the Data Warehouse**__

Let see the overview from the data that we can get from the data warehouse

---

In [10]:
date = pd.read_sql_table("dim_date", conn, schema=schema)
time = pd.read_sql_table("dim_time", conn, schema=schema)
user = pd.read_sql_table("dim_user", conn, schema=schema)
product = pd.read_sql_table("dim_product", conn, schema=schema)
seller = pd.read_sql_table("dim_seller", conn, schema=schema)
feedback = pd.read_sql_table("dim_feedback", conn, schema=schema)
fct_order_item = pd.read_sql_table("fct_order_items", conn, schema=schema)

In [11]:
# Collections for each dataset
datasets = [date, time, user, product, seller, feedback, fct_order_item]
names = ['date', 'time', 'user', 'product', 'seller', 'feedback', 'order']

# Creating a DataFrame with useful information about all datasets
data_info = pd.DataFrame({})
data_info['dataset'] = names
data_info['n_rows'] = [df.shape[0] for df in datasets]
data_info['n_cols'] = [df.shape[1] for df in datasets]
data_info['null_amount'] = [df.isnull().sum().sum() for df in datasets]
data_info['qty_null_columns'] = [len([col for col, null in df.isnull().sum().items() if null > 0]) for df in datasets]
data_info['null_columns'] = [', '.join([col for col, null in df.isnull().sum().items() if null > 0]) for df in datasets]

data_info.style.background_gradient()

,dataset,n_rows,n_cols,null_amount,qty_null_columns,null_columns
0,date,2191,15,0,0,
1,time,1440,6,0,0,
2,user,96096,6,0,0,
3,product,32951,11,1838,7,"product_name_length, product_description_length, product_photos_qty, product_length_cm, product_weight_g, product_height_cm, product_width_cm"
4,seller,3095,6,0,0,
5,feedback,98854,8,0,0,
6,order,112650,36,7852,10,"order_approved_date, order_approved_time, pickup_date, pickup_time, delivered_date, delivered_time, num_payment, total_payment_value, total_payment_installment, lifetime_order"


## Datamart 1

---
Order related datamart

In [4]:
QUERY = """
-- Data Mart 1 
-- Sales Trend Analysis Datamart
select 
	foi.order_id ,
	foi.order_item_id ,
	u.user_name,
	p.product_id ,
	p.product_category ,
	foi.price,
	foi.shipping_cost ,
	foi.total_payment_value 
from staging.fct_order_items foi
left outer join staging.dim_date dd on foi.order_date = dd.date_id 
left outer join staging.dim_time dt on foi.order_time = dt.time_id 
left join (
	select 
		du.user_key ,
		du.user_name
	from staging.dim_user du 
	where du.is_current_version=true
) u on foi.user_key = u.user_key
left join (
	select 
		dp.product_key ,
		dp.product_id , 
		dp.product_category 
	from staging.dim_product dp 
	where dp.is_current_version=true
) p on foi.product_key = p.product_key;
"""

In [5]:
# Init dataframe
order = pd.read_sql_query(QUERY, conn)

In [6]:
order.head().style.background_gradient()

,order_id,order_item_id,user_name,product_id,product_category,price,shipping_cost,total_payment_value
0,00018f77f2f0320c557190d7a144bdd3,1,eb28e67c4c0b83846050ddfb8a35d051,e5f2d52b802189ee658865ca93d83a8f,pet_shop,239900.000000,19930.000000,259830.000000
1,00024acbcdf0a6daa1e931b038114c75,1,af861d436cfc08b2c2ddefd0ba074622,7634da152a4610f1595efa32f14722fc,perfumery,12990.000000,12790.000000,25780.000000
2,00042b26cf59d7ce69dfabb4e55b4fd9,1,64b576fb70d441e8f1b2d7d446e483c5,ac6c3623068f30de03045865e4e10089,garden_tools,199900.000000,18140.000000,218040.000000
3,00048cc3ae777c65dbb7d2a0634bc1ea,1,85c835d128beae5b4ce8602c491bf385,ef92defde845ab8450f9d70c526ef70f,housewares,21900.000000,12690.000000,34590.000000
4,0005a1a1728c9d785b8e2b08b904576c,1,639d23421f5517f69d0c3d6e6564cf0e,310ae3c140ff94b03219ad0adc3c778f,health_beauty,145950.000000,11650.000000,157600.000000


### Understanding Data 

In [ ]:
# Data preparation module loading
from dataprep.eda import create_report, plot_correlation, plot_missing, plot

# Understanding distribution
plot(order)

In [ ]:
# Finding & Handling Missing Value
plot_missing(order)

In [ ]:
# So, columns with missing value : product_category, total_payment_value -> need to find way to handle it

# First, need to see what is the prob
# Product_category

plot(order.loc[order.product_category.isna()])

In [ ]:
# Distribution not much diff, can handle using flag
order['product_category'].fillna("Other", inplace=True)

In [ ]:
order.loc[order.total_payment_value.isna()]

In [ ]:
# Only 1 item, could handle it manually
payment_val = (44990.0 + 2830.0) * 3
order.loc[order.total_payment_value.isna(), 'total_payment_value'] = payment_val

In [ ]:
# To summarize : 

def handle_missing_mart(df):
    df['product_category'].fillna("Other", inplace=True)
    payment_val = (44990.0 + 2830.0) * 3
    df.loc[df.total_payment_value.isna(), 'total_payment_value'] = payment_val
    
    return df

In [ ]:
# Reporting Result

from dataprep.eda import create_report
report = create_report(order, title='Item Transaction Data Mart')

report

## Datamart 2

---
User related datamart

In [ ]:
QUERY = """
-- RFM Analysis Datamart
-- Churn Analysis Datamart
-- CLTV = ((Average Order Value x Purchase Frequency)/Churn Rate) x Profit margin. -> churn rate + purchase frequency hitung manual nanti
select 
	u.user_name,
	EXTRACT(DAY from '2018-09-03'::timestamp - MAX(dd."date")) as recency,
	MAX(foi.lifetime_order) as frequency ,
	MAX(foi.lifetime_spending) as monetary,
	MAX(dd."date") - MIN(dd."date") as usage_days,
	MAX(foi.lifetime_spending) / MAX(foi.lifetime_order) as average_order_value,
	MAX(foi.lifetime_order) <= 1 as isChurned
from staging.fct_order_items foi
left outer join staging.dim_date dd on foi.order_date = dd.date_id 
left outer join (
	select 
		du.user_key ,
		du.user_name 
	from staging.dim_user du 
	where du.is_current_version=true
) u on foi.user_key = u.user_key
group by u.user_name 
order by 5 desc;
"""

In [ ]:
# Init dataframe
user = pd.read_sql_query(QUERY, conn)
user.info()

### Understanding Data 

In [ ]:
# Understanding distribution
plot(user)

In [ ]:
# Finding & Handling Missing Value
plot_missing(user)

In [ ]:
# Understanding correlation 

plot_correlation(user)

In [ ]:
report = create_report(user, title='User Data Mart')
report

## Datamart 3

---
Product related datamart

In [ ]:
QUERY = """
-- Data mart no aggregation
select 
	foi.order_id ,
	p.product_id,
	p.product_category,
    foi.price,
	foi.total_payment_value 
from staging.fct_order_items foi 
left outer join (
	select 
		dp.product_key ,
		dp.product_id ,
		dp.product_category
	from staging.dim_product dp 
	where dp.is_current_version=true
) p on foi.product_key = p.product_key
order by 1,2;
"""

In [ ]:
# Init dataframe
prod = pd.read_sql_query(QUERY, conn)
prod.info()

### Understanding Data 

In [ ]:
plot(prod)

In [ ]:
plot_missing(prod)

In [ ]:
prod_cleaned = handle_missing_mart(prod)
prod_cleaned.info()

In [ ]:
plot(prod_cleaned)